In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import ElasticNet

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

import joblib

In [2]:
df = pd.read_csv("../../DATA/Advertising.csv")

In [3]:
df.head()

,TV,radio,newspaper,sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,9.3
3,151.5,41.3,58.5,18.5
4,180.8,10.8,58.4,12.9


In [4]:
df.describe()

,TV,radio,newspaper,sales
count,200.000000,200.000000,200.000000,200.000000
mean,147.042500,23.264000,30.554000,14.022500
std,85.854236,14.846809,21.778621,5.217457
min,0.700000,0.000000,0.300000,1.600000
25%,74.375000,9.975000,12.750000,10.375000
50%,149.750000,22.900000,25.750000,12.900000
75%,218.825000,36.525000,45.100000,17.400000
max,296.400000,49.600000,114.000000,27.000000


In [5]:
X = df.drop('sales', axis=1)
y = df['sales']

In [6]:
## Train | Validation | Hold out Test
## 70%    | 15%         | 15%

In [7]:
X_train, X_other, y_train, y_other = train_test_split(X, y, test_size=0.3, random_state=101)

In [8]:
X_eval, X_test, y_eval, y_test = train_test_split(X_other, y_other, test_size=0.5, random_state=101)

In [9]:
## Scale the data

In [10]:
scaler = StandardScaler() 

In [11]:
X_train = scaler.fit_transform(X_train)
X_eval = scaler.transform(X_eval)
X_test = scaler.transform(X_test)

In [12]:
base_elasticnet_model = ElasticNet()

In [13]:
param_grid = {'alpha':[.1, 1, 5, 10, 50, 100],
              'l1_ratio': [.1, .5, .7, .9, .95, .99, 1]}

In [14]:
grid_model = GridSearchCV(estimator=base_elasticnet_model,
                          param_grid=param_grid,
                          scoring='neg_root_mean_squared_error',
                          cv=10,verbose=2)

In [15]:
grid_model.fit(X_train, y_train)

Fitting 10 folds for each of 42 candidates, totalling 420 fits
[CV] END ............................alpha=0.1, l1_ratio=0.1; total time=   0.0s
[CV] END ............................alpha=0.1, l1_ratio=0.1; total time=   0.0s
[CV] END ............................alpha=0.1, l1_ratio=0.1; total time=   0.0s
[CV] END ............................alpha=0.1, l1_ratio=0.1; total time=   0.0s
[CV] END ............................alpha=0.1, l1_ratio=0.1; total time=   0.0s
[CV] END ............................alpha=0.1, l1_ratio=0.1; total time=   0.0s
[CV] END ............................alpha=0.1, l1_ratio=0.1; total time=   0.0s
[CV] END ............................alpha=0.1, l1_ratio=0.1; total time=   0.0s
[CV] END ............................alpha=0.1, l1_ratio=0.1; total time=   0.0s
[CV] END ............................alpha=0.1, l1_ratio=0.1; total time=   0.0s
[CV] END ............................alpha=0.1, l1_ratio=0.5; total time=   0.0s
[CV] END ............................alpha=0.1

GridSearchCV(cv=10, estimator=ElasticNet(),
             param_grid={'alpha': [0.1, 1, 5, 10, 50, 100],
                         'l1_ratio': [0.1, 0.5, 0.7, 0.9, 0.95, 0.99, 1]},
             scoring='neg_root_mean_squared_error', verbose=2)

In [16]:
grid_model.best_params_

{'alpha': 0.1, 'l1_ratio': 1}

In [17]:
## Lasso is the best model

In [18]:
y_eval_pred = grid_model.predict(X_eval)

In [19]:
print("RMSE on evaluation : ", np.sqrt(mean_squared_error(y_eval, y_eval_pred)))

RMSE on evaluation :  1.5716450447699666


In [20]:
y_test_pred = grid_model.predict(X_test)

In [21]:
## Final Performance Metric
print("RMSE on test set :", np.sqrt(mean_squared_error(y_test, y_test_pred)))

RMSE on test set : 1.5180965507585331


In [22]:
## Final Model

In [23]:
final_model = ElasticNet(alpha=0.1, l1_ratio=1) 

In [24]:
## Scale the feature dataset before feeding to the final model
X = scaler.fit_transform(X)

In [25]:
final_model.fit(X,y)

ElasticNet(alpha=0.1, l1_ratio=1)

In [26]:
joblib.dump(scaler,'scaler.pkl')
joblib.dump(final_model, 'final_model.pkl')

['final_model.pkl']

In [27]:
## loading the model

In [28]:
loaded_model = joblib.load("final_model.pkl") 

In [29]:
campaign = [[230.1, 37.8, 69.2]]

In [30]:
scaled_campaign = scaler.transform(campaign)

/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [31]:
scaled_campaign

array([[0.96985227, 0.98152247, 1.77894547]])

In [32]:
loaded_model.predict(scaled_campaign)

array([20.37046666])